In [1]:
# Zelle 1
# =============
# Hier laden wir die wichtigsten Bibliotheken und stellen sicher,
# dass wir im richtigen Environment sind.

# Wichtigste Imports
import os
import json
import torch

# Für ONNX-Export:
import onnx
import onnxruntime


import ray.cloudpickle as pickle
import torch


from onnxruntime.quantization import quantize_static, CalibrationDataReader, QuantType, CalibrationMethod
import onnxruntime.quantization.calibrate as calibrate
import numpy as np
import cv2


# (Optional) Für FP16-Konvertierung oder INT8-Quantisierung:
# Hier kann man z.B. onnxruntime-extensions / PyTorch FX Graph Mode etc. nutzen.
# Zunächst installieren wir ggf. onnxruntime-tools, falls wir quantisieren wollen.
# Das kann man in der requirements.txt so angeben:
# onnxruntime
# onnx
# torch
# numpy
# psutil   # falls wir später GPU/CPU usage tracken
# onnxruntime-tools  # optional für quantization utilities

# Kontrolle über das Device:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

initial_onnx_transform = False


Using device: cuda


In [2]:
def export_to_onnx(
    torch_model: torch.nn.Module,
    output_filename: str,
    input_shape=(1, 3, 520, 520),
    dynamic_batch: bool=True,
    opset_version: int=13
) -> None:
    # Bestimme das Device aus dem ersten Parameter des Modells
    # (funktioniert nur, wenn das Modell mindestens einen Parameter hat!)
    model_device = next(torch_model.parameters()).device

    # 1) Lege einen Dummy-Input an (float32)
    dummy_input = torch.randn(*input_shape, dtype=torch.float32, device=model_device)

    if dynamic_batch:
        dynamic_axes = {
            'input': {0: 'batch_size'},
            'output': {0: 'batch_size'}
        }
    else:
        dynamic_axes = {}

    # 2) Export nach ONNX
    torch.onnx.export(
        model=torch_model,
        args=dummy_input,
        f=output_filename,
        export_params=True,
        opset_version=opset_version,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['output'],
        dynamic_axes=dynamic_axes,
    )

    print(f"ONNX-Export abgeschlossen. Gespeichert unter: {output_filename}")
    if dynamic_batch:
        print(" --> Batch-Dimension ist dynamisch.")
    else:
        print(" --> Feste Batch-Dimension im ONNX-Graph.")


In [3]:


# Beispiel: Falls deine TrainedModel-Klasse in Helper/ml_models.py liegt:
from Helper.ml_models import TrainedModel

# Pfad zur JSON mit den Checkpoints
best_checkpoints_json = "/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/FINAL_DATEN/best_checkpoints.json"

# 1) JSON laden
with open(best_checkpoints_json, "r") as f:
    best_checkpoints = json.load(f)

print("Gefundene Modelle in JSON:", list(best_checkpoints.keys()))


# 2) Hilfsfunktion zum Laden eines PyTorch-Modells aus checkpoint.pkl
def load_pytorch_model(model_name: str, checkpoint_path: str, 
                       width=2048, height=1024, 
                       skip_local_load=True) -> TrainedModel:
    """
    Erstellt ein TrainedModel-Objekt und lädt die Ray-Tune Checkpointdaten.
    """
    # Wir legen irgendein Dummy-Folder und Dummy-Gewichtsname an, 
    # damit die Klasse nicht meckert.
    dummy_folder_path = "/tmp/onnx_export_temp"
    dummy_weights_name = "temp_weights"

    # 1) Erzeuge das TrainedModel (skip_local_load=True -> kein .pth-Laden)
    model_obj = TrainedModel(
        model_name=model_name,
        width=width,
        height=height,
        weights_name=dummy_weights_name,
        folder_path=dummy_folder_path,
        start_epoch="latest",
        skip_local_load=skip_local_load
    )
    
    # 2) Ray-Checkpoint laden (per pickle)
    if not os.path.isfile(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found at: {checkpoint_path}")
    with open(checkpoint_path, "rb") as fp:
        checkpoint_data = pickle.load(fp)
    
    # 3) State_dicts wiederherstellen
    model_obj.model.load_state_dict(checkpoint_data["model_state"])
    if "optimizer_state" in checkpoint_data:
        model_obj.optimizer.load_state_dict(checkpoint_data["optimizer_state"])
    
    model_obj.model.eval()
    model_obj.model.to(device)  # auf CPU oder GPU
    return model_obj


if initial_onnx_transform:
    for arch_name, ckp_path in best_checkpoints.items():
        print(f"\n--- Start Export für {arch_name} ---")
        # 1) Laden
        model_trained = load_pytorch_model(arch_name, ckp_path)
        
        # 2) Export nach ONNX
        onnx_filename = f"{arch_name}.onnx"
        export_to_onnx(
            torch_model=model_trained.model, 
            output_filename=f'/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/ONNX/{onnx_filename}',
            input_shape=(1, 3, 520, 520),  # oder (1,3,H,W)
            dynamic_batch=True,           # True oder False
            opset_version=13
        )
        
        # 3) Modell entladen, GPU-Cache leeren
        del model_trained
        torch.cuda.empty_cache()
        print(f"Export und Entladen von {arch_name} abgeschlossen.\n")




2025-01-29 14:15:05.361707: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-29 14:15:05.910331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Gefundene Modelle in JSON: ['lraspp_mobilenet_v3_large', 'fcn_resnet101', 'deeplabv3_resnet50', 'fcn_resnet50', 'deeplabv3_mobilenet_v3_large', 'deeplabv3_resnet101']


In [4]:
# Für FP16-Konvertierung
# (Installiere vorher: pip install onnxruntime-extensions onnxruntime-tools)
try:
    from onnxconverter_common import float16
    HAS_FLOAT16_CONVERTER = True
except ImportError:
    HAS_FLOAT16_CONVERTER = False
    print("[WARN] float16_converter nicht verfügbar. FP16-Konvertierung wird ggf. übersprungen.")

# Für INT8-Dynamic Quantization
try:
    from onnxruntime.quantization import quantize_dynamic, QuantType
    HAS_QUANT_DYNAMIC = True
except ImportError:
    HAS_QUANT_DYNAMIC = False
    print("[WARN] quantize_dynamic nicht verfügbar. Dynamische INT8-Quantisierung wird ggf. übersprungen.")


def convert_fp16_onnx(fp32_onnx_path: str, fp16_onnx_path: str) -> None:
    """
    Konvertiert ein ONNX-Modell von FP32 nach FP16 mit onnxconverter-common.float16.
    """
    if not os.path.isfile(fp32_onnx_path):
        print(f"[ERROR] Datei existiert nicht: {fp32_onnx_path}")
        return
    
    try:
        print(f"Starte FP16-Konvertierung von: {fp32_onnx_path}")

        # Lade das ONNX-Modell
        model = onnx.load(fp32_onnx_path)

        # Wandle auf FP16 um
        model_fp16 = float16.convert_float_to_float16(model)

        # Speichere das FP16-Modell
        onnx.save(model_fp16, fp16_onnx_path)
        print(f"✅ FP16-Modell gespeichert unter: {fp16_onnx_path}")

    except Exception as e:
        print(f"[ERROR] FP16-Konvertierung fehlgeschlagen: {e}")


def quantize_int8_dynamic(fp32_onnx_path: str, int8_onnx_path: str) -> None:
    """
    Dynamische INT8-Quantisierung mit onnxruntime.quantization.quantize_dynamic.
    Achtung: Eher einfache, heuristische Quantisierung ohne Kalibrierung.
    """
    if not HAS_QUANT_DYNAMIC:
        print("[ERROR] quantize_dynamic nicht verfügbar.")
        return
    
    if not os.path.isfile(fp32_onnx_path):
        print(f"[ERROR] Datei existiert nicht: {fp32_onnx_path}")
        return
    
    try:
        print(f"Starte dynamische INT8-Quantisierung: {fp32_onnx_path}")
        # Liste der Ops, die quantisiert werden sollen
        # Typischerweise: ['Conv', 'MatMul', 'Gemm'] 
        # (je nach Modell ggf. mehr)
        op_types_to_quantize = ['Conv', 'MatMul']

        quantize_dynamic(
            model_input=fp32_onnx_path,
            model_output=int8_onnx_path,
            op_types_to_quantize=op_types_to_quantize,
            weight_type=QuantType.QUInt8  # oder QuantType.QInt8
        )
        print(f"INT8-Modell gespeichert unter: {int8_onnx_path}")
    except Exception as e:
        print(f"[ERROR] Dynamische INT8-Quantisierung schlug fehl: {e}")


In [5]:
# class CalibrationDataLoader(CalibrationDataReader):
#     """
#     Custom DataLoader für ONNX INT8-Kalibrierung.
#     Lädt Bilder aus einem Ordner, skaliert sie auf das Modellinputformat und gibt sie als Batch zurück.
#     """

#     def __init__(self, calibration_data_path: str, input_tensor_name: str, target_shape=(1, 3, 520, 520)):
#         """
#         :param calibration_data_path: Pfad zum Ordner mit den Kalibrierungsbildern
#         :param input_tensor_name: Name des Eingabetensors für ONNX
#         :param target_shape: Zielgröße der Bilder (Batch, Channels, Height, Width)
#         """
#         self.image_paths = [os.path.join(calibration_data_path, f) 
#                             for f in os.listdir(calibration_data_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
#         self.input_tensor_name = input_tensor_name
#         self.target_shape = target_shape
#         self.index = 0
#         print(f"Gefundene Kalibrierbilder: {len(self.image_paths)}")
        
#         if len(self.image_paths) == 0:
#             raise ValueError("[ERROR] Keine Bilder für die Kalibrierung gefunden!")

#     def get_next(self):
#         """
#         Lädt das nächste Batch an Bildern, skaliert sie und gibt sie zurück.
#         """
#         if self.index >= len(self.image_paths):
#             return None  # Ende der Daten

#         batch_images = []
#         for _ in range(self.target_shape[0]):  # Batch-Größe
#             if self.index >= len(self.image_paths):
#                 break
            
#             img_path = self.image_paths[self.index]
#             image = cv2.imread(img_path)  # Lade Bild mit OpenCV
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konvertiere zu RGB
#             image = cv2.resize(image, (self.target_shape[3], self.target_shape[2]))  # Resize auf (Width, Height)
#             image = image.astype(np.float32) / 255.0  # Normalisieren auf [0,1]
#             image = np.transpose(image, (2, 0, 1))  # Channel-First Format (C, H, W)
#             batch_images.append(image)
#             self.index += 1

#         batch_images = np.array(batch_images).astype(np.float32)  # In NumPy Array umwandeln
#         batch_images = np.expand_dims(batch_images, axis=0)  # Batch-Dimension hinzufügen
#         return {self.input_tensor_name: batch_images}

#     def rewind(self):
#         """
#         Setzt den Index zurück, um die Daten erneut zu laden.
#         """
#         self.index = 0



# def quantize_int8_calibrated(fp32_onnx_path: str, int8_onnx_path: str, calibration_data_path: str, input_tensor_name: str):
#     """
#     Führt eine INT8-Quantisierung mit Kalibrierung durch.
#     """
#     # Lade Kalibrierungs-Dataloader
#     data_reader = CalibrationDataLoader(
#         calibration_data_path=calibration_data_path,
#         input_tensor_name=input_tensor_name,
#         target_shape=(1, 3, 520, 520)  # Modell-Inputgröße setzen
#     )

#     print(f"🚀 Starte INT8-Kalibrierung für {fp32_onnx_path}...")
    
#     quantize_static(
#         model_input=fp32_onnx_path,
#         model_output=int8_onnx_path,
#         calibration_data_reader=data_reader,
#     )
    
#     print(f"✅ INT8-Modell gespeichert unter: {int8_onnx_path}")

# # Beispielaufruf:
# quantize_int8_calibrated(
#     fp32_onnx_path="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/ONNX/deeplabv3_resnet50.onnx",
#     int8_onnx_path="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/ONNX/deeplabv3_resnet50_int8.onnx",
#     calibration_data_path="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/",
#     input_tensor_name="input"  
# )



In [6]:
import os
import cv2
import numpy as np
import torch
import gc
import random
from onnxruntime.quantization import quantize_static, CalibrationDataReader

class CalibrationDataLoader(CalibrationDataReader):
    """
    Custom DataLoader für ONNX INT8-Kalibrierung mit Speicheroptimierung.
    Verarbeitet eine Stichprobe von Bildern, anstatt alle zu nutzen.
    """

    def __init__(self, calibration_data_path: str, input_tensor_name: str, target_shape=(3, 520, 520), max_samples=300):
        """
        :param calibration_data_path: Pfad zum Ordner mit den Kalibrierungsbildern
        :param input_tensor_name: Name des Eingabetensors für ONNX
        :param target_shape: Zielgröße der Bilder (Channels, Height, Width)
        :param max_samples: Maximal zu verwendende Bilder für Kalibrierung (z. B. 300)
        """
        all_images = [
            os.path.join(calibration_data_path, f)
            for f in os.listdir(calibration_data_path) if f.endswith(('.png', '.jpg', '.jpeg'))
        ]

        # **Stichprobe der Bilder wählen (zufällig oder jedes X-te Bild)**
        self.image_paths = random.sample(all_images, min(len(all_images), max_samples))
        
        self.input_tensor_name = input_tensor_name
        self.target_shape = target_shape
        self.index = 0
        self.total_images = len(self.image_paths)

        print(f"🔍 Nutze {self.total_images} Bilder für die Kalibrierung (statt {len(all_images)})")

        if self.total_images == 0:
            raise ValueError("[ERROR] Keine Bilder für die Kalibrierung gefunden!")

        self.use_cuda = torch.cuda.is_available()
        if self.use_cuda:
            print("🚀 CUDA/GPU-Beschleunigung wird genutzt!")
        else:
            print("⚠️ Keine CUDA-fähige GPU gefunden. Fallback auf CPU.")

    def get_next(self):
        """
        Lädt das nächste Bild, skaliert es und gibt es zurück (speicheroptimiert).
        """
        if self.index >= self.total_images:
            print("✅ Kalibrierung abgeschlossen.")
            return None  # Ende der Daten

        img_path = self.image_paths[self.index]
        image = cv2.imread(img_path)  # Lade Bild mit OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Konvertiere zu RGB
        image = cv2.resize(image, (self.target_shape[2], self.target_shape[1]))  # Resize auf (Width, Height)
        image = image.astype(np.float32) / 255.0  # Normalisieren auf [0,1]
        image = np.transpose(image, (2, 0, 1))  # Channel-First Format (C, H, W)

        # Falls GPU verfügbar ist, lade das Bild auf die GPU
        if self.use_cuda:
            image = torch.tensor(image).cuda()

        batch_image = np.expand_dims(image.cpu().numpy(), axis=0)  # Auf CPU verschieben, NumPy Format setzen

        # 🚀 Speicherverwaltung: Lösche nicht mehr benötigte Variablen
        del image
        self.index += 1  # Zähler erhöhen
        print(f"🔹 Verarbeite Bild {self.index}/{self.total_images}: {img_path}")

        # **Wichtig:** Nach jedem Bild explizit Speicher freigeben!
        torch.cuda.empty_cache()  # 🧹 GPU-Speicher leeren
        gc.collect()  # 🧹 Garbage Collector aufrufen

        return {self.input_tensor_name: batch_image}  # ✅ Rank 4 (1, C, H, W)

    def rewind(self):
        """
        Setzt den Index zurück und startet die Kalibrierung erneut.
        """
        self.index = 0
        print("🔄 Kalibrierung wird erneut gestartet")


def quantize_int8_calibrated(fp32_onnx_path: str, int8_onnx_path: str, calibration_data_path: str, input_tensor_name: str, max_samples=300):
    """
    Führt eine INT8-Quantisierung mit Kalibrierung durch (mit Speicheroptimierung).
    """
    # Lade Kalibrierungs-Dataloader mit reduzierter Anzahl an Bildern
    data_reader = CalibrationDataLoader(
        calibration_data_path=calibration_data_path,
        input_tensor_name=input_tensor_name,
        target_shape=(3, 520, 520),  # Modell-Inputgröße setzen
        max_samples=max_samples  # **Nutze nur X Bilder**
    )

    print(f"🚀 Starte INT8-Kalibrierung für {fp32_onnx_path}...")

    quantize_static(
        model_input=fp32_onnx_path,
        model_output=int8_onnx_path,
        calibration_data_reader=data_reader,
    )

    print(f"✅ INT8-Modell gespeichert unter: {int8_onnx_path}")


# **🛠️ Beispielaufruf:**
quantize_int8_calibrated(
    fp32_onnx_path="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/ONNX/deeplabv3_resnet50.onnx",
    int8_onnx_path="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/ONNX/deeplabv3_resnet50_int8.onnx",
    calibration_data_path="/home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/",
    input_tensor_name="input",
    max_samples=300  # **Nur 300 Bilder nutzen**
)


🔍 Nutze 300 Bilder für die Kalibrierung (statt 3475)
🚀 CUDA/GPU-Beschleunigung wird genutzt!
🚀 Starte INT8-Kalibrierung für /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/ONNX/deeplabv3_resnet50.onnx...


🔹 Verarbeite Bild 1/300: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/0000099_01.png
🔹 Verarbeite Bild 2/300: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/000002624_01.png
🔹 Verarbeite Bild 3/300: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/000001978_01.png
🔹 Verarbeite Bild 4/300: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/000002248_01.png
🔹 Verarbeite Bild 5/300: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/000003326_01.png
🔹 Verarbeite Bild 6/300: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/000001825_01.png
🔹 Verarbeite Bild 7/300: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/CityscapesDaten/images/000001760_01.png
🔹 Verarbeite Bild 8/300: /home/jan/studienarbeit/Studienarbeit-CODE_Sem

✅ INT8-Modell gespeichert unter: /home/jan/studienarbeit/Studienarbeit-CODE_Semantische_Segmentation/ONNX/deeplabv3_resnet50_int8.onnx
